In [1]:
# Import necessary libraries

# Images
import cv2
import numpy as np
from pathlib import Path
import mediapipe as mp
import matplotlib.pyplot as plt

# PDF's
import fitz

In [2]:
# Define paths
input_dir = Path("photo/images")
pdf_dir = Path("pdfs")
output_dir_face = Path("face_detected_MediaPipe")
cropped_dir_face = Path("cropped_faces_MediaPipe")
pdf_cropped_dir = Path("pdf_cropped_faces_MediaPipe")

output_dir_face.mkdir(parents=True, exist_ok=True)
cropped_dir_face.mkdir(parents=True, exist_ok=True)
pdf_cropped_dir.mkdir(parents=True, exist_ok=True)

In [3]:
# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7)

In [4]:
def detect_faces_mediapipe(img, img_name, cropped_dir, annotated_dir):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_detection.process(img_rgb)

    face_count = 0
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = img.shape
            x1 = int(bboxC.xmin * w)
            y1 = int(bboxC.ymin * h)
            x2 = int((bboxC.xmin + bboxC.width) * w)
            y2 = int((bboxC.ymin + bboxC.height) * h)

            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face_crop = img[y1:y2, x1:x2]
            face_count += 1
            face_filename = cropped_dir / f"{img_name}_face_{face_count}.jpg"
            cv2.imwrite(str(face_filename), face_crop)
            print(f"Saved: {face_filename.name}")

            # Draw rectangle on the face
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    annotated_path = annotated_dir / f"{img_name}_annotated.jpg"
    cv2.imwrite(str(annotated_path), img)
    print(f"Annotated image saved: {annotated_path.name}")

In [5]:
def pdf_to_images(pdf_path, dpi=300):
    pdf_document = fitz.open(str(pdf_path))
    img_paths = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        zoom = dpi / 72
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)
        img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
        img_paths.append((img_array, page_num + 1))
    return img_paths

In [6]:
def process_images(max_images=25):
    image_counter = 0
    for folder in input_dir.iterdir():
        if not folder.is_dir():
            continue
        for img_path in folder.glob("*.jpg"):
            if image_counter >= max_images:
                print(f"Reached the max limit of {max_images} images.")
                return
            img = cv2.imread(str(img_path))
            detect_faces_mediapipe(img, img_path.stem, cropped_dir_face, output_dir_face)
            image_counter += 1

In [7]:
def process_pdfs():
    pdf_files = list(pdf_dir.glob("*.pdf"))
    for pdf_file in pdf_files:
        print(f"Processing PDF: {pdf_file.name}")
        try:
            pages = pdf_to_images(pdf_file)
            for img_array, page_num in pages:
                img_name = f"{pdf_file.stem}_page_{page_num}"
                detect_faces_mediapipe(img_array, img_name, pdf_cropped_dir, output_dir_face)
        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")

In [8]:
process_images(max_images=25)

Annotated image saved: 00_annotated.jpg
Annotated image saved: 01_annotated.jpg
Annotated image saved: 02_annotated.jpg
Annotated image saved: 03_annotated.jpg
Annotated image saved: 04_annotated.jpg
Annotated image saved: 05_annotated.jpg
Annotated image saved: 06_annotated.jpg
Annotated image saved: 07_annotated.jpg
Annotated image saved: 08_annotated.jpg
Annotated image saved: 09_annotated.jpg
Annotated image saved: 10_annotated.jpg
Annotated image saved: 11_annotated.jpg
Annotated image saved: 12_annotated.jpg
Annotated image saved: 13_annotated.jpg
Annotated image saved: 14_annotated.jpg
Annotated image saved: 15_annotated.jpg
Annotated image saved: 16_annotated.jpg
Annotated image saved: 17_annotated.jpg
Annotated image saved: 18_annotated.jpg
Annotated image saved: 19_annotated.jpg
Annotated image saved: 20_annotated.jpg
Annotated image saved: 21_annotated.jpg
Annotated image saved: 22_annotated.jpg
Annotated image saved: 23_annotated.jpg
Annotated image saved: 24_annotated.jpg


In [9]:
process_pdfs()

Processing PDF: formato_prueba_famoso_1.pdf
Annotated image saved: formato_prueba_famoso_1_page_1_annotated.jpg
Processing PDF: formato_prueba_famoso_2.pdf
Annotated image saved: formato_prueba_famoso_2_page_1_annotated.jpg
Processing PDF: formato_prueba_famoso_3.pdf
Annotated image saved: formato_prueba_famoso_3_page_1_annotated.jpg
Processing PDF: formato_prueba_famoso_diferente_1.pdf
Annotated image saved: formato_prueba_famoso_diferente_1_page_1_annotated.jpg


# MediaPipe

In [2]:
# Import necessary libraries
import os
import time
from pathlib import Path
from typing import List, Tuple, Optional

# Images
import cv2
import numpy as np
from pathlib import Path
import mediapipe as mp
import matplotlib.pyplot as plt

# PDF's
import fitz

In [ ]:
# Face detection using MediaPipe
class FaceDetector:
    # Configuration
    def __init__(self):
        self.input_dir = ("photo/images")
        self.ouput_dir_mediapipe("face_detected_MediaPipe")
        self.cropped_dir_face = Path("cropped_faces_MediaPipe")
        self.no_face_detected: List[Path] = []

        